# Analiza pomiarów

Analiza i wizualizacja danych zebranych przy uruchamianiu algorytmu Knutha oraz algorytmu ewolucyjnego z różnymi parametrami.

In [48]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path
from glob import glob
import re
from collections import defaultdict

from typing import Tuple, List, Dict

## Pomocnicze funkcje

In [99]:
def read_data_files(directory: str) -> Tuple[List[str], Dict[str, List[str]]]:
    files = glob(f'{directory}/*.csv')
    
    names = []
    data = defaultdict(list)
    for file_path in files:
        names.append(Path(file_path).stem)
        with open(file_path, 'r') as file:
            line = ''
            while(not re.match(r'Parametry', line)):
                line = file.readline()

            line = file.readline()
            parts = line.split(';')
            while len(parts) == 2:
                    data[parts[0]].append(parts[1].strip())

                    line = file.readline()
                    parts = line.split(';')

            while(not re.match(r'Wyniki', line)):
                    line = file.readline()

            line = file.readline()
            parts = line.split(';')
            while len(parts) == 2:
                data[parts[0]].append(parts[1].strip())

                line = file.readline()
                parts = line.split(';')
    
    tmp = data['Najmniejsza liczba generacji']
    data['Najmniejsza liczba generacji'] = data['Największa liczba generacji']
    data['Największa liczba generacji'] = tmp
    
    return names, data

In [100]:
def read_data_files_to_df(directory: str, index: str = None) -> pd.DataFrame:
    index_name = Path(directory).stem
    names, data = read_data_files(directory)
    
    df = pd.DataFrame(data)
    if (index):
        df.set_index(index)
    else:
        df.index = pd.Index(names, name=index_name)
    
    df = df.astype(
        {'Liczba symulacji': int, 
         'Wielkość populacji': int, 
         'Procent osobników używanych do reprodukcji': int,
         'Prawdopodobieństwo krzyżowania': float,
         'Prawdopodobieństwo mutacji': float,
         'Makymalna liczba pokoleń': int,
         'Średnia liczba populacji': float,
         'Liczba zabitych populacji': int,
         'Najmniejsza liczba generacji': int,
         'Największa liczba generacji': int
        })
    
    return df

## Algorytm Knutha

In [53]:
# TODO: Jakaś prosta analiza jedych wyników

## Algorytm gentyczny

Analizy wyników dla różnych parametrów.

In [80]:
base_path = 'Wyniki/Algorytm_ewolucyjny/Pojedyncze_zmiany'

### Metoda selekcji

In [104]:
df = read_data_files_to_df(f'{base_path}/metoda_selekcji')
results_df = df.iloc[:, -4:]
results_df

,Średnia liczba populacji,Liczba zabitych populacji,Najmniejsza liczba generacji,Największa liczba generacji
metoda_selekcji,,,,
ruletka,5.79,4,3,13
prosta,5.07,1,1,13
losowa,8.04,32,1,13
turniej,5.84,10,3,13


In [124]:
x = 'Metoda selekcji'
y = 'Średnia liczba populacji'
px.bar(df.sort_values(y), x=x, y=y, color=x, 
       title='Wpływ metody selekcji na średnią liczbę ruchów')